# Assembler rubustness - Breadth of Coverage

## Imports

In [45]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np
from statistics import mean

## Global variables

In [43]:
METRICS_COLUMNS_GLOBAL = ['run','sample','assembler', 'contigs', 'basepairs', 'max_contig', 'Ns', 'n50', 'misassemblies', 'mapped_reads']
ASSEMBLER_PROCESS_LIST = ["BCALM2", "GATBMINIAPIPELINE", "MINIA", "MEGAHIT", "METASPADES", "UNICYCLER", "SPADES",
                          "SKESA", "VELVETOPTIMIZER", "IDBA"]
PROCESS_TO_NAME = {"BCALM2": "BCALM2", 
                   "GATBMINIAPIPELINE": "GATBMiniaPipeline",
                   "MINIA": "MINIA",
                   "MEGAHIT": "MEGAHIT", 
                   "METASPADES": "metaSPAdes", 
                   "UNICYCLER": "Unicycler", 
                   "SPADES": "SPAdes",
                    "SKESA": "SKESA",
                   "VELVETOPTIMIZER": "VelvetOptimizer",
                   "IDBA": "IDBA-UD"}
log_distributed = ['LHS', 'LNN', 'ERR2935805']
even_distribution = ['EHS', 'LNN', 'ERR2984773']
COLOURS = ['#004B93', "#009392", "#39B185", "#9CCB86", "#E9E29C", "#EEB479", "#E88471","#CF597E",'lightgray', 'darkgray']

## Load data

In [17]:
_files = glob.glob('../Results/*/results/*/stats/*_breadth_of_coverage_contigs.csv')

df_list = []
for f in _files:
    run = f.split('/')[-5]
    sample = f.split('/')[-3]
    assembler = f.split('/')[-1].split('_')[1]
    df = pd.read_csv(f)
    df['run'] = run
    df['Sample'] = sample
    df['Assembler'] = assembler
    df_list.append(df)

df = pd.concat(df_list)
df['distribution'] = np.where(df['Sample'].isin(log_distributed), 'Log', 'Even')
df

,Reference,Breadth of Coverage,Contigs,run,Sample,Assembler,distribution
0,Bacillus_subtilis,0.976840,115,run1,LNN,BCALM2,Log
1,Enterococcus_faecalis,0.973012,165,run1,LNN,BCALM2,Log
2,Escherichia_coli_plasmid,0.955394,5,run1,LNN,BCALM2,Log
3,Escherichia_coli,0.790588,1120,run1,LNN,BCALM2,Log
4,Lactobacillus_fermentum,0.917745,235,run1,LNN,BCALM2,Log
...,...,...,...,...,...,...,...
7,Salmonella_enterica,0.976461,72,run2,ERR2984773,Unicycler,Even
8,Staphylococcus_aureus,0.982598,27,run2,ERR2984773,Unicycler,Even
9,Staphylococcus_aureus_plasmid1,1.000000,1,run2,ERR2984773,Unicycler,Even
10,Staphylococcus_aureus_plasmid2,1.000000,1,run2,ERR2984773,Unicycler,Even


In [66]:
len(df["Reference"].unique())

12

## Plot data

In [85]:
for sample in sorted(df['Sample'].unique(), key=lambda v: v.upper(), reverse=True):
    print('-' + sample)
    fig=make_subplots(rows=6, cols=2, column_titles=sorted(df['Reference'].unique(), key=lambda v: v.upper(), reverse=True),
                     shared_yaxes=True, shared_xaxes=True, x_title="Contigs", y_title="Breadth of Coverage")
    row=1
    col=1
    for reference in sorted(df['Reference'].unique(), key=lambda v: v.upper(), reverse=True):
        print('--' + reference)
        i=0
        print(row, col)
        for assembler in sorted(df['Assembler'].unique(), key=lambda v: v.upper(), reverse=True):
            print('---' + assembler)
            showlegend=True if col==1 and row==1 else False
            contigs = list(df['Contigs'][(df['Assembler'] == assembler) & (df['Reference'] == reference) & (df['Sample'] == sample)])
            boc = list(df['Breadth of Coverage'][(df['Assembler'] == assembler) & (df['Reference'] == reference) & (df['Sample'] == sample)])
            if contigs: #list not empty
                fig.add_trace(go.Scatter(x=[mean(contigs)],
                                         y=[mean(boc)],
                                         error_y=dict(type='data', # value of error bar given in data coordinates
                                                      symmetric=False,
                                                      array=[max(boc)-mean(boc)],
                                                      arrayminus=[min(boc)-mean(boc)],
                                                      visible=True),
                                         error_x=dict(type='data', # value of error bar given in data coordinates
                                                      symmetric=False,
                                                      array=[max(contigs)-mean(contigs)],
                                                      arrayminus=[min(contigs)-mean(contigs)],
                                                      visible=True),
                                         name=assembler, marker=dict(color=COLOURS[i]), showlegend=showlegend),
                              col=col, row=row)
            i+=1
        if col == 2:
            col = 1
            row += 1
        else:
            col += 1
    fig.update_layout(plot_bgcolor='rgb(255,255,255)', title="{} Genome Coverage variation".format(sample))
    fig.update_xaxes(showline=True, linewidth=1, linecolor='#DCDCDC', gridcolor='#DCDCDC', rangemode='tozero')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='#DCDCDC', gridcolor='#DCDCDC', range=[0, 1])
    for i in fig['layout']['annotations']:
        i['font']['size'] = 12
    fig.show()
    plot(fig, filename='Plots/Genome Fragmentation/{}.html'.format(sample), auto_open=False)

-LNN
--Staphylococcus_aureus_plasmid3
1 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid2
1 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid1
2 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus
2 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Salmonella_enterica
3 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Pseudomonas_aeruginosa
3 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Listeria_monocytogenes
4 1
---VelvetO

-LHS
--Staphylococcus_aureus_plasmid3
1 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid2
1 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid1
2 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus
2 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Salmonella_enterica
3 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Pseudomonas_aeruginosa
3 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Listeria_monocytogenes
4 1
---VelvetO

-ERR2984773
--Staphylococcus_aureus_plasmid3
1 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid2
1 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid1
2 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus
2 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Salmonella_enterica
3 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Pseudomonas_aeruginosa
3 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Listeria_monocytogenes
4 1
---

-ERR2935805
--Staphylococcus_aureus_plasmid3
1 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid2
1 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid1
2 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus
2 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Salmonella_enterica
3 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Pseudomonas_aeruginosa
3 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Listeria_monocytogenes
4 1
---

-ENN
--Staphylococcus_aureus_plasmid3
1 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid2
1 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid1
2 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus
2 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Salmonella_enterica
3 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Pseudomonas_aeruginosa
3 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Listeria_monocytogenes
4 1
---VelvetO

-EHS
--Staphylococcus_aureus_plasmid3
1 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid2
1 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus_plasmid1
2 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Staphylococcus_aureus
2 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Salmonella_enterica
3 1
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Pseudomonas_aeruginosa
3 2
---VelvetOptimizer
---Unicycler
---SPAdes
---SKESA
---MINIA
---metaSPAdes
---MEGAHIT
---IDBA-UD
---GATBMiniaPipeline
---BCALM2
--Listeria_monocytogenes
4 1
---VelvetO